Installations & imports

In [1]:
!pip install -q pypdf sentence-transformers faiss-cpu transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 49.8 MB/s eta 0:00:00


In [2]:
import os
import json
import re
from pathlib import Path
from dataclasses import dataclass, asdict
from typing import List, Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Core libraries
import numpy as np
import pandas as pd
import torch

# RAG-specific libraries
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import faiss

Configurations

In [3]:
class RAGConfig:
    """Configuration settings for the RAG system"""

    # Paths
    DOCS_DIR = Path("./docs")

    # Models
    EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
    QA_MODEL = "distilbert-base-uncased-distilled-squad"
    GENERATION_MODEL = "google/flan-t5-base"  # Change to flan-t5-base for faster processing

    # Chunking parameters
    CHUNK_SIZE = 500
    CHUNK_OVERLAP = 100

    # Retrieval parameters
    TOP_K_CHUNKS = 3
    MAX_NEW_TOKENS = 400

    # File constraints
    MAX_PDFS = 3
    MAX_TOTAL_MB = 30

config = RAGConfig()

Data structuring

In [4]:
@dataclass
class PageRecord:
    """Represents a single page from a PDF"""
    doc_id: str          # filename without extension
    source_path: str     # full file path
    page_num: int        # 1-based page number
    text: str           # cleaned text content
    char_count: int     # number of characters

@dataclass
class ChunkRecord:
    """Represents a text chunk for embedding"""
    doc_id: str
    source_path: str
    page_num: int
    chunk_id: int        # chunk number within the page
    text: str
    char_count: int
    word_count: int

pdf processing class

In [5]:
class PDFProcessor:
    """Handles PDF loading, text extraction, and cleaning"""

    @staticmethod
    def setup_docs_directory():
        """Create docs directory if it doesn't exist"""
        config.DOCS_DIR.mkdir(parents=True, exist_ok=True)
        print(f"📁 Documents directory ready: {config.DOCS_DIR.resolve()}")
        return config.DOCS_DIR

    @staticmethod
    def clean_text(text: str) -> str:
        """Clean and normalize text"""
        if not text:
            return ""

        # Remove null characters and normalize whitespace
        text = text.replace("\x00", " ")
        text = re.sub(r"[ \t]+", " ", text)  # Collapse spaces/tabs
        text = re.sub(r"\n\s*\n", "\n\n", text)  # Normalize line breaks
        text = re.sub(r"\f", "\n", text)  # Replace form feeds with newlines

        return text.strip()

    @staticmethod
    def extract_pdf_pages(pdf_path: Path) -> List[PageRecord]:
        """Extract text from all pages of a PDF"""
        try:
            reader = PdfReader(str(pdf_path))
            pages = []

            for i, page in enumerate(reader.pages):
                raw_text = page.extract_text() or ""
                cleaned_text = PDFProcessor.clean_text(raw_text)

                pages.append(PageRecord(
                    doc_id=pdf_path.stem,
                    source_path=str(pdf_path),
                    page_num=i + 1,
                    text=cleaned_text,
                    char_count=len(cleaned_text)
                ))

            print(f"📄 Extracted {len(pages)} pages from {pdf_path.name}")
            return pages

        except Exception as e:
            print(f"❌ Error processing {pdf_path.name}: {e}")
            return []

    @staticmethod
    def load_all_pdfs() -> List[PageRecord]:
        """Load and process all PDFs in the docs directory"""
        pdf_paths = sorted(config.DOCS_DIR.glob("*.pdf"))

        if not pdf_paths:
            print(f"❌ No PDFs found in {config.DOCS_DIR.resolve()}")
            print("Please add 2-3 PDF files to this directory.")
            return []

        # Validate PDF count and size
        n_pdfs = len(pdf_paths)
        total_mb = sum(p.stat().st_size for p in pdf_paths) / (1024 * 1024)

        print(f"\n📋 Found {n_pdfs} PDF(s):")
        for p in pdf_paths:
            size_kb = p.stat().st_size / 1024
            print(f"   • {p.name} ({size_kb:.1f} KB)")

        if n_pdfs > config.MAX_PDFS:
            print(f"⚠️  Warning: {n_pdfs} PDFs found, but only processing first {config.MAX_PDFS}")
            pdf_paths = pdf_paths[:config.MAX_PDFS]

        if total_mb > config.MAX_TOTAL_MB:
            print(f"⚠️  Warning: Total size {total_mb:.2f} MB exceeds recommended {config.MAX_TOTAL_MB} MB")

        # Process all PDFs
        all_pages = []
        for pdf_path in pdf_paths:
            pages = PDFProcessor.extract_pdf_pages(pdf_path)
            all_pages.extend(pages)

        # Summary statistics
        total_pages = len(all_pages)
        empty_pages = sum(1 for p in all_pages if len(p.text) == 0)
        total_chars = sum(p.char_count for p in all_pages)

        print(f"\n📊 Processing Summary:")
        print(f"   • Total pages: {total_pages}")
        print(f"   • Empty pages: {empty_pages}")
        print(f"   • Total characters: {total_chars:,}")
        print(f"   • Average chars per page: {total_chars/total_pages:.0f}")

        return all_pages

Text chunking class

In [6]:
class TextChunker:
    """Handles text chunking for optimal embedding"""

    @staticmethod
    def chunk_text(text: str, chunk_size: int = config.CHUNK_SIZE,
                   overlap: int = config.CHUNK_OVERLAP) -> List[str]:
        """Split text into overlapping chunks"""
        words = text.split()
        if len(words) <= chunk_size:
            return [text]

        chunks = []
        start = 0

        while start < len(words):
            end = start + chunk_size
            chunk_words = words[start:end]
            chunk_text = " ".join(chunk_words)
            chunks.append(chunk_text)

            if end >= len(words):
                break

            start = end - overlap

        return chunks

    @staticmethod
    def create_chunks_from_pages(pages: List[PageRecord]) -> List[ChunkRecord]:
        """Convert pages to chunks with metadata"""
        all_chunks = []

        for page in pages:
            if not page.text.strip():  # Skip empty pages
                continue

            text_chunks = TextChunker.chunk_text(page.text)

            for chunk_id, chunk_text in enumerate(text_chunks):
                chunk_record = ChunkRecord(
                    doc_id=page.doc_id,
                    source_path=page.source_path,
                    page_num=page.page_num,
                    chunk_id=chunk_id,
                    text=chunk_text,
                    char_count=len(chunk_text),
                    word_count=len(chunk_text.split())
                )
                all_chunks.append(chunk_record)

        # Summary statistics
        print(f"\n🔄 Chunking Summary:")
        print(f"   • Total chunks created: {len(all_chunks)}")
        print(f"   • Average chunk size: {np.mean([c.word_count for c in all_chunks]):.1f} words")
        print(f"   • Min/Max chunk size: {min(c.word_count for c in all_chunks)}/{max(c.word_count for c in all_chunks)} words")

        return all_chunks

RAG system class

In [7]:
class RAGSystem:
    """Complete RAG system with embedding, retrieval, and generation"""

    def __init__(self):
        self.embedding_model = None
        self.qa_model = None
        self.generation_model = None
        self.tokenizer = None
        self.index = None
        self.chunks = []
        self.embeddings = None

    def initialize_models(self):
        """Load all required models"""
        print("🚀 Initializing RAG System...")

        # 1. Embedding model
        print("   🧠 Loading embedding model...")
        self.embedding_model = SentenceTransformer(config.EMBEDDING_MODEL)

        # 2. QA model for extractive answers
        print("   🎯 Loading QA model...")
        self.qa_model = pipeline("question-answering", model=config.QA_MODEL)

        # 3. Generation model
        print("   📝 Loading generation model...")
        self.tokenizer = AutoTokenizer.from_pretrained(config.GENERATION_MODEL)
        self.generation_model = AutoModelForSeq2SeqLM.from_pretrained(config.GENERATION_MODEL)

        # Move to GPU if available
        if torch.cuda.is_available():
            self.generation_model = self.generation_model.to("cuda")
            print("   ✅ Using CUDA acceleration")
        else:
            print("   ℹ️  Using CPU (consider GPU for faster generation)")

        print("✅ All models loaded successfully!")

    def build_knowledge_base(self, chunks: List[ChunkRecord]):
        """Build embeddings and FAISS index"""
        print("\n🔍 Building Knowledge Base...")

        self.chunks = chunks
        chunk_texts = [chunk.text for chunk in chunks]

        # Generate embeddings
        print("   🧠 Generating embeddings...")
        self.embeddings = self.embedding_model.encode(
            chunk_texts,
            show_progress_bar=True,
            convert_to_numpy=True,
            normalize_embeddings=True,
            batch_size=32
        )

        # Build FAISS index
        print("   🔧 Building FAISS index...")
        dim = self.embeddings.shape[1]
        self.index = faiss.IndexFlatIP(dim)  # Inner product for cosine similarity
        self.index.add(self.embeddings.astype('float32'))

        print(f"✅ Knowledge base ready!")
        print(f"   • Embeddings shape: {self.embeddings.shape}")
        print(f"   • Index size: {self.index.ntotal} vectors")

    def retrieve_chunks(self, query: str, k: int = config.TOP_K_CHUNKS) -> List[Tuple[ChunkRecord, float]]:
        """Retrieve most relevant chunks for a query"""
        if not self.index or not self.chunks:
            raise ValueError("Knowledge base not initialized. Call build_knowledge_base() first.")

        # Generate query embedding
        query_embedding = self.embedding_model.encode(
            [query],
            convert_to_numpy=True,
            normalize_embeddings=True
        )

        # Search
        k = min(k, len(self.chunks))  # Ensure k is valid
        scores, indices = self.index.search(query_embedding, k)

        # Return chunks with scores
        results = []
        for idx, score in zip(indices[0], scores[0]):
            if 0 <= idx < len(self.chunks):  # Validate index
                results.append((self.chunks[idx], float(score)))

        return results

    def get_extractive_answers(self, query: str, chunks: List[ChunkRecord]) -> List[Dict]:
        """Get extractive answers using QA model"""
        answers = []

        for chunk in chunks:
            try:
                result = self.qa_model(question=query, context=chunk.text)
                answers.append({
                    "answer": result["answer"],
                    "confidence": float(result["score"]),
                    "doc_id": chunk.doc_id,
                    "page_num": chunk.page_num,
                    "chunk_id": chunk.chunk_id,
                    "context_preview": chunk.text[:200] + "..."
                })
            except Exception as e:
                print(f"⚠️  QA failed for chunk from {chunk.doc_id}: {e}")
                continue

        return sorted(answers, key=lambda x: x["confidence"], reverse=True)

    def generate_answer(self, query: str, retrieved_chunks: List[ChunkRecord]) -> str:
        """Generate comprehensive answer using retrieved context"""
        if not retrieved_chunks:
            return "I couldn't find relevant information to answer your question."

        # Build context with citations
        context_parts = []
        for chunk in retrieved_chunks:
            citation = f"[Document: {chunk.doc_id}, Page: {chunk.page_num}]"
            context_parts.append(f"{citation}\n{chunk.text}")

        full_context = "\n\n".join(context_parts)

        # Create prompt
        prompt = f"""You are a knowledgeable AI assistant. Answer the question using ONLY the provided context.

Instructions:
- Write a comprehensive answer in 2-3 paragraphs
- Cite sources using the format (Document: name, Page: number)
- Explain concepts clearly and thoroughly
- If the information isn't in the context, say so clearly

Context:
{full_context}

Question: {query}

Answer:"""

        # Generate
        try:
            inputs = self.tokenizer(
                prompt,
                return_tensors="pt",
                truncation=True,
                max_length=2048
            )

            # Move to same device as model
            device = next(self.generation_model.parameters()).device
            inputs = inputs.to(device)

            outputs = self.generation_model.generate(
                **inputs,
                max_new_tokens=config.MAX_NEW_TOKENS,
                min_new_tokens=80,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                no_repeat_ngram_size=3,
                repetition_penalty=1.1,
                early_stopping=True,
                pad_token_id=self.tokenizer.eos_token_id
            )

            generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Clean output (remove prompt)
            if "Answer:" in generated_text:
                generated_text = generated_text.split("Answer:")[-1].strip()

            return generated_text

        except Exception as e:
            print(f"❌ Generation failed: {e}")
            return f"I encountered an error generating the answer: {str(e)}"

    def answer_question(self, query: str, k: int = config.TOP_K_CHUNKS, include_extractive: bool = True) -> Dict:
        """Complete RAG pipeline - retrieve and generate answer"""
        print(f"\n🔍 Processing question: '{query}'")

        try:
            # Step 1: Retrieve relevant chunks
            print("   📚 Retrieving relevant information...")
            retrieved = self.retrieve_chunks(query, k)

            if not retrieved:
                return {
                    "query": query,
                    "error": "No relevant information found",
                    "status": "failed"
                }

            chunks = [chunk for chunk, score in retrieved]
            scores = [score for chunk, score in retrieved]

            print(f"   ✅ Retrieved {len(chunks)} chunks (similarity: {scores[0]:.3f}-{scores[-1]:.3f})")

            # Step 2: Get extractive answers (optional)
            extractive_answers = []
            if include_extractive:
                print("   🎯 Extracting specific answers...")
                extractive_answers = self.get_extractive_answers(query, chunks)

            # Step 3: Generate comprehensive answer
            print("   📝 Generating comprehensive answer...")
            generated_answer = self.generate_answer(query, chunks)

            # ✅ Step 4: Add inline citation list at the end
            citations = {
                f"(Doc: {chunk.doc_id}, Page: {chunk.page_num})" for chunk in chunks
            }
            citation_text = " | ".join(sorted(citations))
            generated_answer_with_citations = f"{generated_answer}\n\n📎 Sources: {citation_text}"

            # Step 5: Compile results
            result = {
                "query": query,
                "generated_answer": generated_answer_with_citations,
                "retrievedWavChunks": len(chunks),
                "similarity_scores": scores,
                "sources": [
                    {
                        "doc_id": chunk.doc_id,
                        "page_num": chunk.page_num,
                        "preview": chunk.text[:200] + "..."
                    }
                    for chunk in chunks
                ],
                "status": "success"
            }

            if extractive_answers:
                result["extractive_answers"] = extractive_answers

            print("   ✅ Answer generated successfully!")
            return result

        except Exception as e:
            error_msg = f"RAG system error: {str(e)}"
            print(f"   ❌ {error_msg}")
            return {
                "query": query,
                "error": error_msg,
                "status": "failed"
            }


Main exectutain pipeline

In [8]:
def main_pipeline():
    """Complete RAG system setup and testing pipeline"""
    print("🚀 Starting Complete RAG System Setup")
    print("=" * 60)

    # Step 1: Setup directory
    docs_dir = PDFProcessor.setup_docs_directory()

    # Step 2: Load PDFs
    print("\n📄 Step 1: Loading PDFs...")
    pages = PDFProcessor.load_all_pdfs()

    if not pages:
        print("❌ No PDFs found. Please add PDF files to the docs directory.")
        return None

    # Step 3: Create chunks
    print("\n🔄 Step 2: Creating chunks...")
    chunks = TextChunker.create_chunks_from_pages(pages)

    # Step 4: Initialize RAG system
    print("\n🤖 Step 3: Initializing RAG system...")
    rag = RAGSystem()
    rag.initialize_models()

    # Step 5: Build knowledge base
    print("\n🔍 Step 4: Building knowledge base...")
    rag.build_knowledge_base(chunks)

    print("\n✅ RAG System Ready!")
    print("=" * 60)

    return rag

def test_rag_system(rag: RAGSystem):
    """Test the RAG system with sample questions"""
    print("\n🧪 Testing RAG System")
    print("=" * 40)

    test_questions = [
        "What is machine learning?",
        "How does artificial intelligence work?",
        "What are the benefits of automation?",
        "Explain neural networks"
    ]

    for i, question in enumerate(test_questions, 1):
        print(f"\n🔍 Test {i}: {question}")
        result = rag.answer_question(question, k=3)

        if result["status"] == "success":
            print(f"✅ Success!")
            print(f"📝 Answer: {result['generated_answer'][:200]}...")
            print(f"📊 Sources: {result['retrievedWavChunks']} chunks from {len(set(s['doc_id'] for s in result['sources']))} documents")
        else:
            print(f"❌ Failed: {result.get('error', 'Unknown error')}")

    print("\n✅ Testing completed!")

Usage examples

In [9]:
def interactive_demo(rag: RAGSystem):
    """Interactive demo for testing questions"""
    print("\n🎯 Interactive RAG Demo")
    print("Enter your questions (type 'quit' to exit)")
    print("-" * 40)

    while True:
        question = input("\n❓ Your question: ").strip()

        if question.lower() in ['quit', 'exit', 'q']:
            print("👋 Goodbye!")
            break

        if not question:
            continue

        result = rag.answer_question(question)

        if result["status"] == "success":
            print(f"\n🤖 Answer:")
            print(result["generated_answer"])
            print(f"\n📊 Retrieved from {result['retrievedWavChunks']} chunks")
        else:
            print(f"\n❌ Error: {result.get('error')}")

In [10]:
from google.colab import files
uploaded = files.upload()
# adding pdf files

Saving awsmp_Matillion_ML_eBook_092019.pdf to awsmp_Matillion_ML_eBook_092019.pdf
Saving Machine Learning for Absolute Beginners.pdf to Machine Learning for Absolute Beginners.pdf


In [13]:
import shutil

for fname in uploaded.keys():
    shutil.move(fname, f"./docs/{fname}")

In [14]:
rag = main_pipeline()

🚀 Starting Complete RAG System Setup
📁 Documents directory ready: /content/docs

📄 Step 1: Loading PDFs...

📋 Found 2 PDF(s):
   • Machine Learning for Absolute Beginners.pdf (11616.3 KB)
   • awsmp_Matillion_ML_eBook_092019.pdf (10014.1 KB)
📄 Extracted 169 pages from Machine Learning for Absolute Beginners.pdf
📄 Extracted 19 pages from awsmp_Matillion_ML_eBook_092019.pdf

📊 Processing Summary:
   • Total pages: 188
   • Empty pages: 2
   • Total characters: 250,133
   • Average chars per page: 1330

🔄 Step 2: Creating chunks...

🔄 Chunking Summary:
   • Total chunks created: 189
   • Average chunk size: 210.3 words
   • Min/Max chunk size: 13/500 words

🤖 Step 3: Initializing RAG system...
🚀 Initializing RAG System...
   🧠 Loading embedding model...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

   ✅ Using CUDA acceleration
✅ All models loaded successfully!

🔍 Step 4: Building knowledge base...

🔍 Building Knowledge Base...
   🧠 Generating embeddings...


Batches:   0%|          | 0/6 [00:00<?, ?it/s]

   🔧 Building FAISS index...
✅ Knowledge base ready!
   • Embeddings shape: (189, 384)
   • Index size: 189 vectors

✅ RAG System Ready!


Queries & questions

In [ ]:
question = "What is machine learning?"
result = rag.answer_question(question)

print("🤖 Answer:")
print(result["generated_answer"])

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🔍 Processing question: 'What is machine learning?'
   📚 Retrieving relevant information...
   ✅ Retrieved 3 chunks (similarity: 0.744-0.685)
   🎯 Extracting specific answers...
   📝 Generating comprehensive answer...
   ✅ Answer generated successfully!
🤖 Answer:
Machine learning is the process of building and training models to process data. In this capacity, your models are learning from your data to make predictions. Artificial intelligence, consequently, uses these learnings to make a computer or technology stack act more human, apply learnings in an automated manner. In addition, because of their interdisciplinary nature, experts from a diverse spectrum of disciplines often define data mining and machine learning differently. This has led to confusion, in addition to a genuine overlap between the two disciplines.

📎 Sources: (Doc: Machine Learning for Absolute Beginners, Page: 10) | (Doc: Machine Learning for Absolute Beginners, Page: 15) | (Doc: awsmp_Matillion_ML_eBook_092019, P

In [15]:
!pip install -q gradio

In [16]:
# Make generation faster & safer in Colab
config.GENERATION_MODEL = "google/flan-t5-base"
config.TOP_K_CHUNKS = 3
config.MAX_NEW_TOKENS = 200

# Ensure docs dir exists
PDFProcessor.setup_docs_directory()

📁 Documents directory ready: /content/docs


PosixPath('docs')

In [17]:
import gradio as gr
from pathlib import Path
from typing import List, Union, Any
import shutil
import torch

# ---- Global state for the app ----
rag_instance = None
kb_is_ready = False

def reset_docs_dir():
    """Clean & recreate ./docs so each build is fresh."""
    if config.DOCS_DIR.exists():
        for f in config.DOCS_DIR.glob("*"):
            try:
                f.unlink()
            except IsADirectoryError:
                shutil.rmtree(f)
    config.DOCS_DIR.mkdir(parents=True, exist_ok=True)

def _to_pathlike(f: Any) -> Path:
    """
    Accepts Gradio's file return (str path or temp obj with .name) and returns Path.
    """
    # gr.File returns a list of strings (paths) in Gradio v4
    if isinstance(f, (str, Path)):
        return Path(f)
    # some environments hand back objects with a .name attr
    if hasattr(f, "name"):
        return Path(f.name)
    raise ValueError(f"Unsupported file type from uploader: {type(f)}")

def save_uploaded_pdfs(files: List[Union[str, Path, Any]]) -> List[Path]:
    """Persist uploaded PDFs to ./docs and return saved paths."""
    saved = []
    if not files:
        return saved
    for f in list(files)[:config.MAX_PDFS]:
        fpath = _to_pathlike(f)
        dest = config.DOCS_DIR / fpath.name
        shutil.copyfile(str(fpath), str(dest))
        saved.append(dest)
    return saved

def build_knowledge_base(files: List[Union[str, Path, Any]]):
    """
    1) Save PDFs to ./docs
    2) Load pages, chunk
    3) Init models (once) + build FAISS
    Returns a status message & quick stats.
    """
    global rag_instance, kb_is_ready

    if not files or len(files) == 0:
        return gr.update(value="❌ Please upload 1–3 PDFs first.")

    # Step 1: save files
    reset_docs_dir()
    saved = save_uploaded_pdfs(files)

    # Step 2: load + chunk
    pages = PDFProcessor.load_all_pdfs()
    if not pages:
        return gr.update(value="❌ Could not read any pages from uploaded PDFs.")

    chunks = TextChunker.create_chunks_from_pages(pages)

    # Step 3: init RAG (only once) + build index
    if rag_instance is None:
        rag = RAGSystem()
        rag.initialize_models()
    else:
        rag = rag_instance

    rag.build_knowledge_base(chunks)

    # Persist globals
    rag_instance = rag
    kb_is_ready = True

    # Status text
    doc_list = "\n".join([f"• {p.name}" for p in saved])
    msg = (
        "✅ Knowledge base built!\n\n"
        f"Uploaded PDFs:\n{doc_list}\n\n"
        f"Chunks in index: {len(chunks)}\n"
        f"Embedding dim: {rag.embeddings.shape[1] if rag.embeddings is not None else 'n/a'}\n"
        f"Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}"
    )
    return gr.update(value=msg)

def format_sources(sources):
    """Nicely render sources with citations."""
    if not sources:
        return "—"
    lines = []
    for s in sources:
        lines.append(f"- **Doc:** `{s['doc_id']}` — **Page:** {s['page_num']} — **Preview:** {s['preview']}")
    return "\n".join(lines)

def ask(query: str, top_k: int, include_extractive: bool):
    """Answer a question using the built KB."""
    global rag_instance, kb_is_ready
    if not query or query.strip() == "":
        return "❌ Please enter a question.", ""
    if not kb_is_ready or rag_instance is None:
        return "❌ Build the knowledge base first.", ""

    try:
        res = rag_instance.answer_question(query=query, k=int(top_k), include_extractive=include_extractive)
    except Exception as e:
        return f"❌ Error: {e}", ""

    if res.get("status") != "success":
        return f"❌ {res.get('error', 'Unknown error')}", ""

    answer_md = res["generated_answer"]
    sources_md = format_sources(res.get("sources"))
    return answer_md, sources_md

def clear_state():
    """Clear global state and wipe docs dir."""
    global rag_instance, kb_is_ready
    rag_instance = None
    kb_is_ready = False
    reset_docs_dir()
    return "🧹 Cleared. Upload PDFs and rebuild the knowledge base."


In [19]:
# --- Gradio App (Blocks) ---
with gr.Blocks(title="Mini RAG (PDF → FAISS → T5)", theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        "# 📚 Mini RAG System\n"
        "Upload 2–3 PDFs → Build Knowledge Base → Ask Questions with citations.\n"
    )

    with gr.Row():
        with gr.Column(scale=1):
            pdfs = gr.File(
                label="Upload PDFs (max 3, total ≤ 30MB)",
                file_count="multiple",
                file_types=[".pdf"]
            )
            build_btn = gr.Button("🔍 Build Knowledge Base", variant="primary")
            status = gr.Markdown("ℹ️ Waiting for PDFs…")
            clear_btn = gr.Button("🧹 Clear & Reset", variant="secondary")

        with gr.Column(scale=1):
            question = gr.Textbox(label="Your Question", placeholder="e.g., What is machine learning?")
            topk = gr.Slider(1, 8, value=config.TOP_K_CHUNKS, step=1, label="Top-K chunks")
            include_extractive = gr.Checkbox(value=True, label="Include extractive answers (DistilBERT)")
            ask_btn = gr.Button("🤖 Ask", variant="primary")
            answer = gr.Markdown(label="Answer")
            sources = gr.Markdown(label="Retrieved Sources")

    # Wire callbacks
    build_btn.click(fn=build_knowledge_base, inputs=[pdfs], outputs=[status])
    ask_btn.click(fn=ask, inputs=[question, topk, include_extractive], outputs=[answer, sources])
    clear_btn.click(fn=clear_state, inputs=None, outputs=[status])

# Launch the app
demo.queue().launch(debug=False, share=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>